In [1]:
import asyncio
import datetime
import time
import pprint

import uvloop
import boto3
from addict import Dict

import uuid

asyncio.set_event_loop_policy(uvloop.EventLoopPolicy())
pretty = pprint.PrettyPrinter(indent=1)

In [2]:
# see https://boto3.readthedocs.io/en/latest/reference/services/dynamodb.html?highlight=dynamodb#service-resource
dydbr = boto3.resource('dynamodb', endpoint_url='http://localhost:8000')

In [3]:
def _Drop(dynamoR, TableName):
    """ Overview: _Drop is a meta function that abstracting away the boto logic
        for each of the particular table operator.
        
        @param dynamoR is a boto3 session resource
        @param TableName is a str-like value naming the table to issues commands about
    """    
    
    try:
        r = dynamoR.meta.client.delete_table(TableName=TableName)
    except Exception as e:
        r = {'TableDescription':{"error":"ResourceNotFoundException","message":"Cannot do operations on a non-existent table"}}
    finally:
        return r

def _Wipe(dynamoR, TableName, CreateFN):
    """ Overview: _Wipe is a meta function that abstracts the similar parts for 
        each of the particular table constructors.
        
        @param dynamoR is a boto3 session resource
        @param TableName is a str-like value naming the table to issues commands about
        @param CreateFN is a function pointer to the create function for the table in question
    """
    waitForGone = dynamoR.meta.client.get_waiter('table_not_exists')
    waitForHere = dynamoR.meta.client.get_waiter('table_exists')
    
    # start delete then wait
    _Drop(dynamoR, TableName)
    waitForGone.wait(TableName=TableName,WaiterConfig={'Delay': 6,'MaxAttempts':10 })
    
    # start create then wait 
    CreateFN(dynamoR)
    waitForHere.wait(TableName=TableName,WaiterConfig={'Delay': 6,'MaxAttempts':10 })
    
    return dynamoR.meta.client.describe_table(TableName=TableName)

In [4]:
def postTableCreate(dynamoR):
    return dynamoR.create_table(
        TableName='Posts',
        AttributeDefinitions=[
            {'AttributeName': '_id',
                'AttributeType': 'S'},
            {'AttributeName': '_loc',
                'AttributeType': 'S'}
        ],
        KeySchema=[
            {'AttributeName': '_id',
                'KeyType': 'HASH'}      
        ],
        GlobalSecondaryIndexes=[
            {   'IndexName': 'ix_LOC',
                'KeySchema': [
                    {'AttributeName': '_loc',
                        'KeyType': 'HASH'},
                ],
                'Projection': {
                    'ProjectionType': 'ALL'
                },
                'ProvisionedThroughput': {
                    'ReadCapacityUnits': 3,
                    'WriteCapacityUnits': 3
                }
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )

def postTableDrop(dynamoR):
    return _Drop(dynamoR, 'Posts')

def postTableWipe(dynamoR):
    """ @param dynamoR is a boto3 client """
    return _Wipe(dynamoR, 'Posts', postTableCreate)

In [5]:
def yearIdxTableCreate(dynamoR):
    return dynamoR.create_table(
        TableName='YearIdx',
        AttributeDefinitions=[
            {'AttributeName': 'yearIdx',
                'AttributeType': 'S'}
        ],
        KeySchema=[
            {'AttributeName': 'yearIdx',
                'KeyType': 'HASH'}
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 3,
            'WriteCapacityUnits': 3
        }
    )
def yearIdxTableDrop(dynamoR):
    return _Drop(dynamoR, 'YearIdx')
def yearIdxTableWipe(dynamoR):
    return _Wipe(dynamoR, 'YearIdx', yearIdxTableCreate)

In [6]:
def monthIdxTableCreate(dynamoR):
    return dynamoR.create_table(
        TableName='MonthIdx',
        AttributeDefinitions=[
            {'AttributeName': 'monthIdx',
                'AttributeType': 'S'}
        ],
        KeySchema=[
            {'AttributeName': 'monthIdx',
                'KeyType': 'HASH'}
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 3,
            'WriteCapacityUnits': 3
        }
    )
def monthIdxTableDrop(dynamoR):
    return _Drop(dynamoR, 'MonthIdx')
def monthIdxTableWipe(dynamoR):
    return _Wipe(dynamoR, 'MonthIdx', monthIdxTableCreate)

In [7]:
def SiteStateTableCreate(dynamoR):
    return dynamoR.create_table(
        TableName='SiteState',
        AttributeDefinitions=[
            {'AttributeName': 'year',
                'AttributeType': 'N'},
            {'AttributeName': 'epoch',
                'AttributeType': 'N'},            
        ],
        KeySchema=[
            {'AttributeName': 'year',
                'KeyType': 'HASH'},
            {'AttributeName': 'epoch',
                'KeyType': 'RANGE'},            
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 3,
            'WriteCapacityUnits': 3
        }
    )
def SiteStateTableDrop(dynamoR):
    return _Drop(dynamoR, 'SiteState')
def SiteStateTableWipe(dynamoR):
    return _Wipe(dynamoR, 'SiteState', SiteStateTableCreate)

In [8]:
# Drop All TablesClear Out
pretty.pprint(yearIdxTableDrop(dydbr)['TableDescription'])
pretty.pprint(monthIdxTableDrop(dydbr)['TableDescription'])
pretty.pprint(postTableDrop(dydbr)['TableDescription'])
pretty.pprint(SiteStateTableDrop(dydbr)['TableDescription'])

{'error': 'ResourceNotFoundException',
 'message': 'Cannot do operations on a non-existent table'}
{'error': 'ResourceNotFoundException',
 'message': 'Cannot do operations on a non-existent table'}
{'error': 'ResourceNotFoundException',
 'message': 'Cannot do operations on a non-existent table'}
{'error': 'ResourceNotFoundException',
 'message': 'Cannot do operations on a non-existent table'}


In [19]:
p = postTableCreate(dydbr)
m = monthIdxTableCreate(dydbr)
y = yearIdxTableCreate(dydbr)
state = SiteStateTableCreate(dydbr)

pretty.pprint(p)
pretty.pprint(m)
pretty.pprint(y)
pretty.pprint(state)

dynamodb.Table(name='Posts')
dynamodb.Table(name='MonthIdx')
dynamodb.Table(name='YearIdx')
dynamodb.Table(name='SiteState')


In [10]:
p = Dict(postTableWipe(dydbr))
m = Dict(monthIdxTableWipe(dydbr))
y = Dict(yearIdxTableWipe(dydbr))
state = Dict(SiteStateTableWipe(dydbr))

print(p.Table.TableName +" "+ p.Table.TableStatus +" "+ str(p.Table.ItemCount))
print(m.Table.TableName +" "+ m.Table.TableStatus +" "+ str(m.Table.ItemCount))
print(y.Table.TableName +" "+ y.Table.TableStatus +" "+ str(y.Table.ItemCount))
print(state.Table.TableName +" "+ state.Table.TableStatus +" "+ str(state.Table.ItemCount))

Posts ACTIVE 0
MonthIdx ACTIVE 0
YearIdx ACTIVE 0
SiteState ACTIVE 0


In [11]:
d = Dict(dydbr.meta.client.list_tables())
d.TableNames

['MonthIdx', 'Posts', 'SiteState', 'YearIdx']

In [20]:
p = Dict(postTableDrop(dydbr))
m = Dict(monthIdxTableDrop(dydbr))
y = Dict(yearIdxTableDrop(dydbr))
state = Dict(SiteStateTableDrop(dydbr))

# lop off the http response meta data  
p = p.TableDescription
m = m.TableDescription
y = y.TableDescription
state = state.TableDescription

# pretty.pprint(p)
# pretty.pprint(m)
# pretty.pprint(y)
# pretty.pprint(state)

print( p.TableName     +' '+ str(p.ItemCount)     +' '+ str(p.TableSizeBytes))
print( m.TableName     +' '+ str(m.ItemCount)     +' '+ str(m.TableSizeBytes))
print( y.TableName     +' '+ str(y.ItemCount)     +' '+ str(y.TableSizeBytes))
print( state.TableName +' '+ str(state.ItemCount) +' '+ str(state.TableSizeBytes))


Posts 0 0
MonthIdx 0 0
YearIdx 0 0
SiteState 0 0
